# Coordinator of a MuSig2 Protocol.

Note: Coordinator could also be a participant.

In [1]:
import json
from aries_askar import Key, KeyAlg
from didcomm_messaging.crypto.backend.askar import AskarCryptoService, AskarSecretKey
from didcomm_messaging.crypto.backend.basic import InMemorySecretsManager
from didcomm_messaging.packaging import PackagingService
from didcomm_messaging.multiformats import multibase
from didcomm_messaging.multiformats import multicodec
from didcomm_messaging.resolver.peer import Peer2, Peer4
from didcomm_messaging.resolver import PrefixResolver
from did_peer_2 import KeySpec, generate, json
from didcomm_messaging import DIDCommMessaging
from didcomm_messaging.routing import RoutingService
from pydid.did import DID


# Define DIDComm Messaging Service Endpoint

In [2]:
didcomm_host = "localhost"
didcomm_port = 8765
didcomm_messaging_uri = f"ws://{didcomm_host}:{didcomm_port}"

# Create Coordinator DID

In [3]:
secrets = InMemorySecretsManager()
crypto = AskarCryptoService()

# verkey = Key.generate(KeyAlg.ED25519)
# verkey_sk_bytes = verkey.get_secret_bytes()
verkey_sk_bytes = b'\xc4\xce4LB/\xf4\xfa-\x1e\x9f\xe7NqT\x9ao\xf1\xb8xI[}`~\xe0\xdfcY\x83<\xae'
verkey = Key.from_secret_bytes(KeyAlg.ED25519, verkey_sk_bytes)


# xkey = Key.generate(KeyAlg.X25519)
# xkey_sk_bytes = xkey.get_secret_bytes()
xkey_sk_bytes = b'\xe9!\xae,\xa8F\xc2y\t\xc6\x86\xcd1\xd4Ys|\xe4\xab\xa8\xe0xx\xe9\xff\x11&\xb3\xbe\x11\xa8\xef'
xkey = Key.from_secret_bytes(KeyAlg.X25519,xkey_sk_bytes)


In [4]:

did = generate(
    [
        KeySpec.verification(
            multibase.encode(
                multicodec.wrap("ed25519-pub", verkey.get_public_bytes()),
                "base58btc",
            )
        ),
        KeySpec.key_agreement(
            multibase.encode(
                multicodec.wrap("x25519-pub", xkey.get_public_bytes()), "base58btc"
            )
        ),
    ],
    [
                {
                    "type": "DIDCommMessaging",
                    "serviceEndpoint": {
                        "uri": didcomm_messaging_uri,
                        "accept": ["didcomm/v2"],
                        "routingKeys": [],
                    },
                },
    ],
)
await secrets.add_secret(AskarSecretKey(verkey, f"{did}#key-1"))
await secrets.add_secret(AskarSecretKey(xkey, f"{did}#key-2"))


In [5]:
did

'did:peer:2.Vz6MksvTpc5zbWrfdZCo3NyjABQQUDqC7yksyBk7PYmCqQ74i.Ez6LSgRMYHNDvqr93XiBp5qNtF1tzQ1cm3gKaoeYukcrnEB1p.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjUiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ'

In [6]:
resolver = PrefixResolver({"did:peer:2": Peer2(), "did:peer:4": Peer4()})

did_document = await resolver.resolve(did)
print(json.dumps(did_document, indent=2))

{
  "@context": [
    "https://www.w3.org/ns/did/v1",
    "https://w3id.org/security/multikey/v1"
  ],
  "id": "did:peer:2.Vz6MksvTpc5zbWrfdZCo3NyjABQQUDqC7yksyBk7PYmCqQ74i.Ez6LSgRMYHNDvqr93XiBp5qNtF1tzQ1cm3gKaoeYukcrnEB1p.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjUiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ",
  "verificationMethod": [
    {
      "type": "Multikey",
      "id": "#key-1",
      "controller": "did:peer:2.Vz6MksvTpc5zbWrfdZCo3NyjABQQUDqC7yksyBk7PYmCqQ74i.Ez6LSgRMYHNDvqr93XiBp5qNtF1tzQ1cm3gKaoeYukcrnEB1p.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjUiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ",
      "publicKeyMultibase": "z6MksvTpc5zbWrfdZCo3NyjABQQUDqC7yksyBk7PYmCqQ74i"
    },
    {
      "type": "Multikey",
      "id": "#key-2",
      "controller": "did:peer:2.Vz6MksvTpc5zbWrfdZCo3NyjABQQUDqC7yksyBk7PYmCqQ74i.Ez6LSgRMYHNDvqr93XiBp5qNtF1tzQ1cm3gKaoeYukcrnEB1p.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjUiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ",

# Instantiate DIDComm Messaging 

This is used to pack and unpack DIDComm messages.

In [7]:
packaging = PackagingService()
router = RoutingService()

dmp = DIDCommMessaging(
    crypto=crypto,
    secrets=secrets,
    resolver=resolver,
    packaging=packaging,
    routing=router,
)

# Create send message functon

Sends packed DIDComm messages to a websocket endpoint

In [8]:
async def send_message(message, to, frm):
    packy = await dmp.pack(
        message=message,
        to=to,
        frm=frm,
    )
    packed = packy.message            
    # Get the first http endpoint from the last DID in the DID chain
    endpoint = packy.get_endpoint("ws")
    async with websockets.connect(endpoint) as websocket:
        message = "Hello from the client!"
        print(f"Sending message: {packed}")
        await websocket.send(packed)  # Send message to the server
        
        response = await websocket.recv()  # Wait for the server's response
        print(f"Received response: {response}")

# Setup DIDComm Messaging Websocket

This is where DIDComm messages are sent to the coordinator

In [9]:
import asyncio
import websockets

inbox = []


# This handler will be called for every client connection.
async def handle_messages(websocket):
    print("Client connected")
    try:
        async for message in websocket:
            print(f"Received DIDComm Message: \n\n {json.dumps(json.loads(message.decode()),indent=2)} \n\n")
            # Echo the received message back to the client.
            # await websocket.send(f"Echo: {message}")

            unpacked =  await dmp.packaging.unpack(crypto, resolver, secrets,message)
            msg = json.loads(unpacked[0].decode())
            inbox.append(msg)
            print(f"Unpacked Message : \n\n {json.dumps(msg, indent=2)} \n\n")
    except websockets.exceptions.ConnectionClosed as e:
        print("Connection closed", e)

# The main function to start the server.
async def run_websocket():
    # Start the server on localhost at port 8765.
    async with websockets.serve(handle_messages, didcomm_host, didcomm_port):
        print(f"WebSocket server started on {didcomm_messaging_uri}")
        # Run the server until it is manually stopped.
        await asyncio.Future()  # Run forever

In [10]:
asyncio.create_task(run_websocket())

<Task pending name='Task-7' coro=<run_websocket() running at /tmp/ipykernel_9458/3993558428.py:24>>

WebSocket server started on ws://localhost:8765
Client connected
Received DIDComm Message: 

 {
  "protected": "eyJ0eXAiOiAiYXBwbGljYXRpb24vZGlkY29tbStlbmNyeXB0ZWQiLCAiYWxnIjogIkVDREgtMVBVK0EyNTZLVyIsICJlbmMiOiAiQTI1NkNCQy1IUzUxMiIsICJhcHUiOiAiWkdsa09uQmxaWEk2TWk1V2VqWk5hM05yYVhCNWIyOHlSMHR5ZW1GSWFFTk1ka3BHVkRaWlkxVlJjVXcxWnpNM1dWUk5hbTFZU0V4WVYzVjNMa1Y2Tmt4VGJrVklXRm8zT1ZsQ1FtUmpXWFJET1hoRlJtbzFkVkl5U21obGFqRXpNak5ZY1hCT1NuZHhia28zWWpNdVUyVjVTakJKYW05cFdrY3dhVXhEU25wSmFuQTNTVzVXZVdGVFNUWkpibVI2VDJrNGRtSkhPV3BaVjNodllqTk9NRTlxWnpOT2FsbHBURU5LYUVscWNHSkpiVkp3V2tkT2RtSlhNSFprYWtscFdGTjNhV05wU1RaWE1URTVabEVqYTJWNUxUSSIsICJhcHYiOiAiN1B2cUNRS0pvbDQwN3pwLW5jeHBKSTZvZmNoQkxBeGJFVVhYSl9COHJEdyIsICJlcGsiOiB7ImNydiI6ICJYMjU1MTkiLCAia3R5IjogIk9LUCIsICJ4IjogIm5ESUQyaUQzUTNjNDBVOVV1VVlLZUh3aWZ0aC1yenY2NjFPeERYNTVRMGsifSwgInNraWQiOiAiZGlkOnBlZXI6Mi5WejZNa3NraXB5b28yR0tyemFIaENMdkpGVDZZY1VRcUw1ZzM3WVRNam1YSExYV3V3LkV6NkxTbkVIWFo3OVlCQmRjWXRDOXhFRmo1dVIySmhlajEzMjNYcXBOSndxbko3YjMuU2V5SjBJam9pWkcwaUxD

# Coordinator Shares DID with participants

Maybe this happens on their website. Or something like that.

In [12]:
print(f"coordinator_did='{did}'")

coordinator_did='did:peer:2.Vz6MksvTpc5zbWrfdZCo3NyjABQQUDqC7yksyBk7PYmCqQ74i.Ez6LSgRMYHNDvqr93XiBp5qNtF1tzQ1cm3gKaoeYukcrnEB1p.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjUiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ'


# Process Inbox for Subscription Requests

In [13]:
import uuid
def create_accept_subscription_message(to):
    print("create_accept_subscription_message")
    msg_id = str(uuid.uuid4())
    msg = {
        "type": "https://didcomm.org/musig2/subscribe/accept",
        "id": msg_id,
        "body": {},
        "from": did,
        "to": to,
    }
    return msg

In [14]:
musig2_subscribers = []

In [15]:
async def process_inbox(inbox):
    while len(inbox) != 0:
        message = inbox.pop(0)
        print("process message", message)
        msg_sender = message["from"]
        if message["type"] == 'https://didcomm.org/musig2/subscribe':
            print("Process subscribe")
            msg = create_accept_subscription_message(msg_sender)
            print(msg)
            asyncio.create_task(send_message(msg, msg_sender, did))
            musig2_subscribers.append(msg_sender)



In [16]:
asyncio.create_task(process_inbox(inbox))


<Task pending name='Task-12' coro=<process_inbox() running at /tmp/ipykernel_9458/391074589.py:1>>

process message {'type': 'https://didcomm.org/musig2/subscribe', 'id': 'e7941053-5bf3-43bc-9054-1c4723ad31c2', 'body': {}, 'from': 'did:peer:2.Vz6Mkskipyoo2GKrzaHhCLvJFT6YcUQqL5g37YTMjmXHLXWuw.Ez6LSnEHXZ79YBBdcYtC9xEFj5uR2Jhej1323XqpNJwqnJ7b3.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjYiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ', 'to': 'did:peer:2.Vz6MksvTpc5zbWrfdZCo3NyjABQQUDqC7yksyBk7PYmCqQ74i.Ez6LSgRMYHNDvqr93XiBp5qNtF1tzQ1cm3gKaoeYukcrnEB1p.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjUiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ'}
Process subscribe
create_accept_subscription_message
{'type': 'https://didcomm.org/musig2/subscribe/accept', 'id': '6ba57cc3-55e2-47fd-b5e4-42189ce85857', 'body': {}, 'from': 'did:peer:2.Vz6MksvTpc5zbWrfdZCo3NyjABQQUDqC7yksyBk7PYmCqQ74i.Ez6LSgRMYHNDvqr93XiBp5qNtF1tzQ1cm3gKaoeYukcrnEB1p.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjUiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ', 'to': 'did:peer:2.Vz6Mkskipyoo2GKrzaHhCLvJFT6YcUQqL5g37YTM

In [ ]:
inbox

In [17]:
musig2_subscribers

['did:peer:2.Vz6Mkskipyoo2GKrzaHhCLvJFT6YcUQqL5g37YTMjmXHLXWuw.Ez6LSnEHXZ79YBBdcYtC9xEFj5uR2Jhej1323XqpNJwqnJ7b3.SeyJ0IjoiZG0iLCJzIjp7InVyaSI6IndzOi8vbG9jYWxob3N0Ojg3NjYiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6W119fQ']

In [18]:
def create_new_cohort_message(to, thread_id):
    print("create_new_cohort_message")
    msg_id = str(uuid.uuid4())
    msg = {
        "type": "https://didcomm.org/musig2/keygen/new_cohort",
        "id": msg_id,
        "thread_id": thread_id,
        "body": {},
        "from": did,
        "to": to,
    }
    return msg

In [19]:
class Musig2Cohort():

    def __init__(self):
        self.id = str(uuid.uuid4())
        self.participants = []
        self.cohort_keys = []

    # def finalize_cohort(self):
        



In [20]:
musig2_cohorts = []

In [21]:
async def announce_new_cohort():
    cohort = Musig2Cohort()
    for subscriber in musig2_subscribers:
        to_array = [subscriber]
        thread_id = cohort.id
        msg = create_new_cohort_message(to_array, thread_id)
        asyncio.create_task(send_message(msg, subscriber, did))
    musig2_cohorts.append(cohort)
                              
        
        

In [22]:
asyncio.create_task(announce_new_cohort())

<Task pending name='Task-15' coro=<announce_new_cohort() running at /tmp/ipykernel_9458/202562631.py:1>>

create_new_cohort_message
Sending message: b'{"protected": "eyJ0eXAiOiAiYXBwbGljYXRpb24vZGlkY29tbStlbmNyeXB0ZWQiLCAiYWxnIjogIkVDREgtMVBVK0EyNTZLVyIsICJlbmMiOiAiQTI1NkNCQy1IUzUxMiIsICJhcHUiOiAiWkdsa09uQmxaWEk2TWk1V2VqWk5hM04yVkhCak5YcGlWM0ptWkZwRGJ6Tk9lV3BCUWxGUlZVUnhRemQ1YTNONVFtczNVRmx0UTNGUk56UnBMa1Y2Tmt4VFoxSk5XVWhPUkhaeGNqa3pXR2xDY0RWeFRuUkdNWFI2VVRGamJUTm5TMkZ2WlZsMWEyTnlia1ZDTVhBdVUyVjVTakJKYW05cFdrY3dhVXhEU25wSmFuQTNTVzVXZVdGVFNUWkpibVI2VDJrNGRtSkhPV3BaVjNodllqTk9NRTlxWnpOT2FsVnBURU5LYUVscWNHSkpiVkp3V2tkT2RtSlhNSFprYWtscFdGTjNhV05wU1RaWE1URTVabEVqYTJWNUxUSSIsICJhcHYiOiAiUTZSTG1vOHVrNHV0V3lkZXpIdnJrdWo4OUZPbDZIbDJNUjBka1ZoTHN0RSIsICJlcGsiOiB7ImNydiI6ICJYMjU1MTkiLCAia3R5IjogIk9LUCIsICJ4IjogImNkVGhsLXp0ZUliMWtjYktoMUxZUUVHUkN4a01iQXJWQVVxRHdCR0hFRm8ifSwgInNraWQiOiAiZGlkOnBlZXI6Mi5WejZNa3N2VHBjNXpiV3JmZFpDbzNOeWpBQlFRVURxQzd5a3N5Qms3UFltQ3FRNzRpLkV6NkxTZ1JNWUhORHZxcjkzWGlCcDVxTnRGMXR6UTFjbTNnS2FvZVl1a2NybkVCMXAuU2V5SjBJam9pWkcwaUxDSnpJanA3SW5WeWFTSTZJbmR6T2k4dmJHOWpZV3hvYjNOME9qZzNO

In [ ]:
inbox